# The Locations clustering POC
The candidate is expected to create a solution - a proof of concept, using Python, to cluster (or simply group) those addresses where they belong to the same location. The candidate is free to use any machine learning model, or hand-crafting solutions to show how the candidate would approach solving this problem.

First, I imported pandas dataframe and read the dataset file.

In [1]:
import pandas as pd
dataset = pd.read_csv("location_data.csv")
dataset

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl
...,...,...,...,...,...
63690,las vegas,meadows ln,4300,89107,nevada
63691,window rock,window rock loop road,264,86515,az
63692,alliance,w state st,2496,44601,oh
63693,denver,s santa fe dr,1395,80223,co


## Exploring the Dataset
Discovering the dataset columns, counting the nulls, and figure out the number of unique values. From the observation, the state column has 121 values, however, the US only has 51 states. Therfore, I need to have a deep look at this column.

In [2]:
print(dataset.columns)
print(dataset.isnull().sum())
dataset.nunique()

Index(['city', 'road', 'house_number', 'postcode', 'state'], dtype='object')
city            0
road            0
house_number    0
postcode        0
state           0
dtype: int64


city             5534
road            17184
house_number     6353
postcode        11015
state             121
dtype: int64

By displaying the state column, I found that some states are written in many formats.

In [3]:
states = set(dataset['state'])
states

{'ak',
 'al',
 'alabama',
 'alaska',
 'ar',
 'arizona',
 'arkansas',
 'az',
 'bc',
 'ca',
 'california',
 'carolina',
 'co',
 'colorado',
 'connecticut',
 'ct',
 'd.c.',
 'daerah khusus ibukota jakarta',
 'dc',
 'de',
 'delaware',
 'district of columbia',
 'fl',
 'florida',
 'ga',
 'georgia',
 'hampshire',
 'hawaii',
 'hi',
 'ia',
 'id',
 'idaho',
 'il',
 'illinois',
 'in',
 'indiana',
 'iowa',
 'island',
 'kansas',
 'kentucky',
 'ks',
 'ky',
 'la',
 'lakes',
 'lombardia',
 'long island',
 'louisiana',
 'ma',
 'maine',
 'maryland',
 'massachusetts',
 'md',
 'me',
 'mi',
 'michigan',
 'minnesota',
 'mississippi',
 'missouri',
 'mn',
 'mo',
 'montana',
 'ms',
 'mt',
 'nb',
 'nc',
 'nd',
 'ne',
 'nebraska',
 'nevada',
 'new hampshire',
 'new jersey',
 'new mexico',
 'new york',
 'nh',
 'nj',
 'nm',
 'north carolina',
 'north dakota',
 'ns',
 'nsw',
 'nv',
 'ny',
 'oh',
 'ohio',
 'ok',
 'oklahoma',
 'on',
 'or',
 'oregon',
 'pa',
 'pennsylvania',
 'range',
 'region',
 'rhode island',
 'ri'

## Data Preprocessing

I did some research and I found an online CSV that contains these values for each state: full name, traditional abbreviation, and USPS.

In [4]:
dataset_state_name = pd.read_csv("states.csv").applymap(str.lower)
dataset_state_name

,full name,traditional abbreviation,usps
0,alabama,ala,al
1,alaska,alas,ak
2,arkansas,ark,ar
3,arizona,ariz,az
4,california,calif,ca
5,colorado,colo,co
6,connecticut,conn,ct
7,delaware,del,de
8,district of columbia,d.c.,dc
9,florida,fla,fl


This for loop is just to map different naming to USPS naming.

In [5]:
for i in range(len(dataset)):
    for j in range(len(dataset_state_name)):
        if dataset['state'][i] == dataset_state_name['full name'][j] or dataset['state'][i] == dataset_state_name['traditional abbreviation'][j] :
            dataset['state'][i] = dataset_state_name['usps'][j]

Checking the number of states again and still, we have an extra 15 observations. 

In [6]:
print(dataset['state'].nunique())
states = set(dataset['state'])
states

66


{'ak',
 'al',
 'ar',
 'az',
 'bc',
 'ca',
 'co',
 'ct',
 'daerah khusus ibukota jakarta',
 'dc',
 'de',
 'fl',
 'ga',
 'hi',
 'ia',
 'id',
 'il',
 'in',
 'island',
 'ks',
 'ky',
 'la',
 'lakes',
 'lombardia',
 'long island',
 'ma',
 'md',
 'me',
 'mi',
 'mn',
 'mo',
 'ms',
 'mt',
 'nb',
 'nc',
 'nd',
 'ne',
 'nh',
 'nj',
 'nm',
 'ns',
 'nsw',
 'nv',
 'ny',
 'oh',
 'ok',
 'on',
 'or',
 'pa',
 'range',
 'region',
 'ri',
 'river south carolina',
 'sc',
 'sd',
 'southwest gulf coast',
 'tn',
 'tri-valley',
 'tx',
 'ut',
 'va',
 'vt',
 'wa',
 'wi',
 'wv',
 'wy'}

As an assumption, I dropped any state name with a length more than the USPS name.

In [7]:
for i in range(len(dataset)):
    if len(dataset['state'][i])!=2:
        dataset = dataset.drop(i)

Now, we have 4 extra state, thats need a deeper review but for now we can leave it.

In [8]:
print(dataset['state'].nunique())
states = set(dataset['state'])
states

55


{'ak',
 'al',
 'ar',
 'az',
 'bc',
 'ca',
 'co',
 'ct',
 'dc',
 'de',
 'fl',
 'ga',
 'hi',
 'ia',
 'id',
 'il',
 'in',
 'ks',
 'ky',
 'la',
 'ma',
 'md',
 'me',
 'mi',
 'mn',
 'mo',
 'ms',
 'mt',
 'nb',
 'nc',
 'nd',
 'ne',
 'nh',
 'nj',
 'nm',
 'ns',
 'nv',
 'ny',
 'oh',
 'ok',
 'on',
 'or',
 'pa',
 'ri',
 'sc',
 'sd',
 'tn',
 'tx',
 'ut',
 'va',
 'vt',
 'wa',
 'wi',
 'wv',
 'wy'}

Importing numby and matplotlib

In [9]:
import numpy as np
import matplotlib.pyplot as plt

Show frequency tables for each categorical feature and counts of unique values

In [10]:
for column in dataset.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=dataset[column],
                        columns='% observations', 
                        normalize='columns'))
    print("# of unique values {}".format(dataset[column].nunique()))

# show summary statistics
display(dataset.describe())

col_0,% observations
city,
'ewa beach,0.000031
10,0.000016
2,0.000016
abbeville,0.000142
abbotsford,0.000142
...,...
zillah,0.000063
zion,0.000079
zion crossroads,0.000016


# of unique values 5523


col_0,% observations
road,
& 166 nwc hwy,0.000063
& 601 highways,0.000016
10 mile rd,0.000079
100th ave ne,0.000063
100th st sw,0.000079
...,...
zelkova ct nw,0.000031
zelkova ct nw inside super walmart,0.000016
zillah w dr,0.000016


# of unique values 17157


col_0,% observations
house_number,
#1651,0.000016
#19,0.000031
#27,0.000016
#38,0.000016
#706,0.000016
...,...
s2107,0.000016
s4255,0.000016
s71w23460,0.000047


...

col_0,% observations
postcode,
01001,0.000079
01007,0.000063
01008,0.000110
01013,0.000047
01013-4820,0.000016
...,...
v3s 9n6,0.000016
v4a 4n3,0.000016
v4p 1a2,0.000016


# of unique values 10990


col_0,% observations
state,
ak,0.001889
al,0.018233
ar,0.012455
az,0.020122
bc,0.000205
ca,0.092929
co,0.015903
ct,0.010093
dc,0.001543


# of unique values 55


,city,road,house_number,postcode,state
count,63511,63511,63511,63511,63511
unique,5523,17157,6350,10990,55
top,houston,main st,101,46227,ca
freq,640,596,357,33,5902


Dropping the duplicates to start the clustering process and then compare the original frequencies and the new ones. It seems that almost 60% of the rows were redundant. 

In [11]:
dataset = dataset.drop_duplicates(ignore_index = True)
dataset
display(dataset.describe())

,city,road,house_number,postcode,state
count,27109,27109,27109,27109,27109
unique,5523,17157,6350,10990,55
top,houston,main st,101,89109,tx
freq,281,207,150,16,2588


## Modeling

State name, City name, and Postcode are unique values that we can use to group the addresses. However, home numbers could be repeated within the same city therefore they couldn't be used as an address. But, If I combined between the house number and the street name, I might get a unique address per home assuming that the street name is the same within the city.

In [12]:
dataset["home adress"] = dataset["house_number"] +" "+ dataset["road"]
dataset = dataset.drop(columns=['house_number', 'road'])
dataset

,city,postcode,state,home adress
0,fort worth,76123,tx,5421 sycamore school rd
1,gautier,39553,ms,2701 highway 90
2,farmington,87401,nm,4750 e main
3,richland,99352,wa,2751 queensgate drive
4,davie,33317,fl,1600 s state hwy seven
...,...,...,...,...
27104,waterloo,50701,ia,2808 university ave
27105,highland,92346,ca,27774 baseline st
27106,window rock,86515,az,264 window rock loop road
27107,denver,80223,co,1395 s santa fe dr


Using the groupby function to cluster the same addresses and print the results.

In [13]:
dataset_grouped = dataset.groupby(['state', 'city', 'postcode'])['home adress'].apply(list)
print(dataset_grouped)

state  city          postcode  
ak     anchorage     99501                                   [3006 mountain view dr]
                     99502         [5000 w international airport rd, 3828 dimond ...
                     99503         [701 e 36th ave, 800 w northern lights blvd, 3...
                     99504         [255 muldoon rd, 1116 muldoon rd, 3035 muldoon...
                     99507                    [2601 e tudor rd, 9100 lake otis pkwy]
                                                         ...                        
wy     rock springs  82901-4020                                        [1607 elk st]
       sheridan      82801         [2146 coffeen ave, 2590 n main st, 2590 n main...
       thermopolis   82443                                         [611 shoshoni st]
       torrington    82240                [800 e valley rd, 800 east valley highway]
       worland       82401                                       [1700 big horn ave]
Name: home adress, Length: 12576,

Yet another fancy way to print all the values. 

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dataset_grouped)

state  city                           postcode  
ak     anchorage                      99501                                   [3006 mountain view dr]
                                      99502         [5000 w international airport rd, 3828 dimond ...
                                      99503         [701 e 36th ave, 800 w northern lights blvd, 3...
                                      99504         [255 muldoon rd, 1116 muldoon rd, 3035 muldoon...
                                      99507                    [2601 e tudor rd, 9100 lake otis pkwy]
                                      99515         [1320 huffman blvd, 8915 old seward hwy, 9100 ...
                                      99517                                         [4350 spenard rd]
       eagle river                    99577                                     [11808 old glenn hwy]
       fairbanks                      99701         [1930 airport way, 449 harold bentley ave, 38 ...
                                 

## Save the pandas series to csv

In [15]:
dataset_grouped.to_csv('output.csv')